In [1]:
%cd /content/drive/MyDrive/python

/content/drive/MyDrive/python


In [2]:
import os
os.getcwd()

'/content/drive/MyDrive/python'

In [ ]:
%%capture
! pip install hdf5storage
! pip install mne==0.23.0
! pip install torch
! pip install PIL
! pip install tensorboardX
! pip install tqdm
! pip install pynwb==2.3.2

In [ ]:
import io
import sys
import socket
import PIL
from torchvision.transforms import ToTensor
from tqdm import tqdm
from gesture.models.d2l_resnet import d2lresnet
from speech_Ruijin.stepping.dataset import dataset_stepping
from speech_Ruijin.stepping.model_stepping import model
if socket.gethostname() == 'workstation':
    sys.path.extend(['C:/Users/wuxiaolong/mydrive/python'])
elif socket.gethostname() == 'LongsMac':
    sys.path.extend(['/Users/long/My Drive/python'])
elif socket.gethostname() == 'DESKTOP-NP9A9VI':
    sys.path.extend(['C:/Users/xiaol/My Drive/python/'])
from datetime import datetime
import pytz
from speech_Ruijin.config import *
from common_dl import *
from torch.utils.data import DataLoader
from speech_Ruijin.dataset import dataset_Dutch
#from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter
set_random_seeds(999)



In [ ]:

dataname='SingleWordProductionDutch'
sid=3
mel_bins=23
sf_EEG=1024

if dataname=='SingleWordProductionDutch':
    pt_folder='sub-' + f"{sid:02d}"
else:
    pt_folder='sid'+str(sid)

resume=False
testing=False
the_time=datetime.now(pytz.timezone('Asia/Shanghai'))
print("Testing: "+str(testing)+".")
if testing or debugging:
    batch_size = 3
    wind_size = 200
    stride = 400
    epoch_num = 1
    patients = 2
    result_dir = data_dir + dataname + '/stepping/' + pt_folder + '/stepping_win_' + str(wind_size) + '_stride_' + str(
        stride) + '/' + the_time.strftime('%Y_%m_%d') + '_' + the_time.strftime('%H_%M') + '_testing/'
else:
    batch_size=128
    wind_size = 200
    stride = 3
    epoch_num = 300
    patients = 20
    result_dir = data_dir + dataname + '/stepping/' + pt_folder + '/stepping_win_' + str(wind_size) + '_stride_' + str(
        stride) + '/' + the_time.strftime('%Y_%m_%d') + '_' + the_time.strftime('%H_%M') + '/'

if not os.path.exists(result_dir):
    print('Result_dir: '+result_dir+'.')
    os.makedirs(result_dir)
writer = SummaryWriter(result_dir)

In [ ]:
stride=10
shift=None #0.03 # -0.03:left shift 30 ms; -30/-60/-90/
#def dataset_stepping(dataname='mydata', sid=1,session=None, pt=1, wind_size=200,stride=1,mel_bins=23,val_len=4000,test_len=4000,shift=None):
train_ds, val_ds, test_ds = dataset_stepping(dataname=dataname,sid=sid,wind_size=wind_size,stride=stride, mel_bins=mel_bins,shift=shift)
# below will read from open dataset
# best is sid=3,4,6,8,9,10
#train_ds, val_ds, test_ds = dataset_Dutch(dataset_name='SingleWordProductionDutch',sid=4,wind_size=wind_size,stride=stride, mel_bins=mel_bins,val_len=4000,test_len=4000,stepping=True)
# Or test on another SEEG audio dataset,sid=[1(mel result looks good),2(TBD),3(TBD)]
#train_ds, val_ds, test_ds = dataset_Dutch(dataset_name='stereoEEG2speech_master',sid=1,highgamma=highgamma,wind_size=wind_size,stride=stride, mel_bins=mel_bins,val_len=4000,test_len=4000)

train_size=len(train_ds)
val_size=len(val_ds)
test_size=len(test_ds)
print('Train size: '+str(train_size)+', val size: '+str(val_size)+', test size: '+str(test_size)+'.')
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=False, pin_memory=False)
val_loader = DataLoader(dataset=val_ds, batch_size=batch_size, shuffle=False, pin_memory=False)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False, pin_memory=False)


In [ ]:
learning_rate=0.005
ch_num=train_ds[0][0].shape[0]
dropout=0.5
#net=model(1000,ch_num,wind_size,6,6,dropout,mel_bins)
net=d2lresnet(task='regression',reg_d=mel_bins)
net=net.to(device)
#optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adagrad(net.parameters(), lr=learning_rate,weight_decay=1e-4)
loss_fun = nn.MSELoss()

In [ ]:
resume_epoch=0
fig,ax=plt.subplots(2,1, figsize=(10,20))
train_losses=[]
val_losses=[]
for epoch in tqdm(range(epoch_num)):
    epoch=epoch+resume_epoch
    print('Training on sid: '+str(sid)+'; Epoch:'+str(epoch)+'.')
    net.train()
    y_preds=[]
    running_loss = 0.0
    for batch, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        x=x.float().to(device)
        y=y.float().to(device)
        y_pred= net(x) # x:torch.Size([3 batch, 118channel, 200time])
        loss = loss_fun(y_pred,y) # y: torch.Size([3, 80])
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.shape[0]
        if batch==len(train_loader)-2:
            plot_y=y
            plot_y_pred=y_pred
        if testing:
            break
    train_loss = running_loss / train_size
    train_losses.append(train_loss)
    #show_batch=min(60,y.shape[0])
    ax[0].imshow(plot_y.cpu().numpy().squeeze().transpose(), cmap='RdBu_r',aspect='auto')
    ax[1].imshow(plot_y_pred.cpu().detach().numpy().squeeze().transpose(), cmap='RdBu_r',aspect='auto')
    fig.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='jpeg')
    buf.seek(0)
    image = PIL.Image.open(buf)
    image = ToTensor()(image).unsqueeze(0)
    writer.add_image('Image/train', image[0], epoch)
    ax[0].clear()
    ax[1].clear()

    print('Evaluating....')
    running_loss = 0.0
    outputs = []
    if epoch % 1 == 0:
        net.eval()
        with torch.no_grad():
            for batch, (val_x, val_y) in enumerate(val_loader):
                val_x=val_x.float().to(device)
                val_y=val_y.float().to(device)
                y_pred = net(val_x)  # x:torch.Size([3 batch, 118channel, 200time])
                loss = loss_fun(y_pred, val_y)  # y: torch.Size([3, 80])

                running_loss += loss.item() * val_x.shape[0]
                if batch==len(val_loader)-2:
                    plot_valy=val_y
                    plot_valy_pred=y_pred
            if testing:
                break
            val_loss = running_loss / val_size
            val_losses.append(val_loss)
        print("Training loss:{:.2f}; Evaluation loss:{:.2f}.".format(train_loss, val_loss))
    ax[0].imshow(plot_valy.cpu().numpy().squeeze().transpose(), cmap='RdBu_r',aspect='auto')
    ax[1].imshow(plot_valy_pred.cpu().detach().numpy().squeeze().transpose(), cmap='RdBu_r',aspect='auto')
    fig.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='jpeg')
    buf.seek(0)
    image = PIL.Image.open(buf)
    image = ToTensor()(image).unsqueeze(0)
    writer.add_image('Image/val', image[0], epoch)
    ax[0].clear()
    ax[1].clear()

    if epoch==resume_epoch:
        best_epoch=resume_epoch
        best_loss=val_loss
        patient=patients
        best_model = {
            'net': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch,
            # 'loss': epoch_loss
        }
    else:
        if val_loss<best_loss:
            best_epoch=epoch
            best_loss=val_loss
            patient=patients
            best_model = {
                'net': net.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': epoch,
                #'loss': epoch_loss
            }

        else:
            patient=patient-1
    print("patients left: {:d}".format(patient))
    if patient==0:
        break
    if testing:
        #pass # test through all epochs
        break

filename=result_dir+'best_model_epoch'+str(best_model['epoch'])+'.pth'
torch.save(best_model,filename)
